In [ ]:
text = "hello I am happy. Are you?"

In [ ]:
# libraries for: regular expressions, file I/O
import re
import sys
import unicodecsv
import liblinearutil as lu

In [ ]:
def preProcess(text):
    #print "original:", text
    # sentence segmentation - assume already done
    pass
    # word tokenisation
    text = re.sub(r"(\w)([.,;:!?'\"”\)])", r"\1 \2", text)
    text = re.sub(r"([.,;:!?'\"“\(])(\w)", r"\1 \2", text)
    #print "tokenising:", text
    tokens = re.split(r"\s+",text)
    # normalisation
    text = re.sub(r"(\S)\1\1+",r"\1\1\1", text)
    tokens = [t.lower() for t in tokens]
    return tokens

In [ ]:
print preProcess(text)

In [ ]:
def ngrams(tokens, max_n):
    features = []
    for i in range(1,max_n+1):
        for j in xrange(0,len(tokens)-(i-1)):
            features.append(" ".join(tokens[j:j+i]))
    return features

In [ ]:
print ngrams(preProcess(text), 2)

In [ ]:
# turn into feature vectors
featureDict = {}
def toFeatureVector(words):
    v = {}
    for w in ngrams(words, 2):
        try:
            i = featureDict[w]
        except KeyError:
            i = len(featureDict) + 1
            featureDict[w] = i
        try:
            v[i] += (1.0/len(words))
        except KeyError:
            v[i] = (1.0/len(words))
    return v

In [ ]:
print toFeatureVector(preProcess(text))

In [ ]:
print featureDict

In [ ]:
# load training data from external files
trainData = []
trainLabels = []
with open('pos.csv', 'rb') as f:
    reader = unicodecsv.reader(f, encoding='utf-8')
    for line in reader:
        trainData.append(toFeatureVector(preProcess(line[0])))
        trainLabels.append(1.0)
with open('neg.csv', 'rb') as f:
    reader = unicodecsv.reader(f, encoding='utf-8')
    for line in reader:
        trainData.append(toFeatureVector(preProcess(line[0])))
        trainLabels.append(-1.0)
print len(featureDict)

In [ ]:
# train
model = lu.train(trainLabels, trainData)

In [ ]:
# predict
def analyseSentiment(text):
    v = toFeatureVector(preProcess(text))
    print v
    return lu.predict([0], [v], model)

In [ ]:
s = analyseSentiment("this is not good")
print "sentiment = ", s